<a href="https://colab.research.google.com/github/FiyinfobaO/used_cars_price_prediction/blob/master/data_scraper_used_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

Cars will be scraped from the [cars45 site](https://buy.cars45.com/cars?limit=100)

In [ ]:
# getting the links to all the cars

urls = []

for i in range(1,43):
  website = "https://buy.cars45.com/cars?limit=100&page={}".format(i)
  page = requests.get(website).text
  soup = bs4(page, 'html.parser')
  for item in soup.find_all(class_='product_box'):
    link = item.a['href']
    urls.append(link)

In [ ]:
# no of cars found
len(urls)

4200

#####Random tests before the main scraping

In [ ]:
for i in urls[:4]:
  test = requests.get(i).text
  s = bs4(test, 'html.parser')

  if s.get_text()[11:16] == 'Sorry':
    print('car is sold out. No details available')
  else:
    a = s.find(class_='price_title amount mt-1').text.strip()
    print(a)
    b = a.replace('₦', '').replace(',', '').lstrip()
    print(b)

₦ 33,000,000
33000000
₦ 850,000
850000
₦ 2,810,000
2810000
₦ 1,858,000
1858000


In [ ]:
test = requests.get('https://buy.cars45.com/cars/toyota-camry-2018-other-ng-760897?limit=100').text
s = bs4(test, 'html.parser')

In [ ]:
s.get_text()[11:16]

'2018 '

In [ ]:
s.find(class_='justify-content-center w-100').text.strip()

'New'

In [ ]:
a = s.find(class_='price_title amount mt-1').string.extract().strip()
print(a)
b = a.replace('₦', '').replace(',', '').lstrip()
b

₦ 13,000,000


'13000000'

In [ ]:
s.find(class_='overview_icon_box clearfix').figcaption.text.strip()

<div class="overview_icon_box clearfix">
<figure>
<img alt="Thumb" src="catalog/view/assets/images/diesel.png"/>
</figure>
<figcaption> Petrol
                                                    </figcaption>
</div>

In [ ]:
na = np.nan
wen = ['sedan', 'suv']
w = s.find(class_='overview_icon_box clearfix body_type')
if w is not None:
  we = w.figcaption.text.strip()
  wen.append(we)
else:
  we = na
  wen.append(we)
print(wen)
de = pd.DataFrame({'wen':wen})


['sedan', 'suv', nan]


In [ ]:
s.findAll('span', class_='float-right')[7]

<span class="float-right">White
                                                            </span>

In [ ]:
aq = ['B', 'D']
aw = s.find_all('div', class_='col-sm-6 col-md-3 col-6')
try:
  awn = aw[4]['title']
  if awn is not None:
    awnn = awn[5]
    aq.append(awnn)
  else:
    awn = np.nan
    aq.append(awn)
except IndexError:
  awn = np.nan
  aq.append(awn)
aq

['B', 'D', nan]

#####Main scraping

In [ ]:
# Getting car information
na = np.nan

manufacturer = []
model = []
year = []
price = []
mileage = []
grade = []
selling_condition = []
transmission = []
color = []
fuel = []

car_err_urls = []

car = 0
for i in tqdm(urls):
  car += 1
  print(f'car {car}')
  page = requests.get(i).text
  soup = bs4(page, 'html.parser')

  # error check to ensure the car isn't sold out
  if soup.get_text()[11:16] == 'Sorry':
    print('car is sold out. No details available')
  else:
    try:
      # price
      pr = soup.find(class_='price_title amount mt-1').text.strip()
      pri = pr.replace('₦', '').replace(',', '').lstrip()
      price.append(pri)
      
      time.sleep(0.2)

      # selling condition
      con = soup.find(class_='justify-content-center w-100')
      if con is not None:
        cond = con.text.strip()
        selling_condition.append(cond)
      else:
        cond = na
        selling_condition.append(cond)

      time.sleep(0.15)

      # fuel
      f = soup.find(class_='overview_icon_box clearfix')
      if f is not None:
        fu = f.figcaption
        if fu is not None:
          fuu = fu.text.strip()
          fuel.append(fuu)
        else:
          fu = na
          fuel.append(fu)
      else:
        fu = na
        fuel.append(fu)
    
      time.sleep(0.15)

      # grade
      g = soup.find_all('div', class_='col-sm-6 col-md-3 col-6')
      try:
        gr = g[3]['title']
        if gr is not None:
          grr = gr[5]
          grade.append(grr)
        else:
          gr = na
          grade.append(gr)
      except IndexError:
        gr = na
        grade.append(gr)
        
      time.sleep(0.15)

      # remaining features
      gen = soup.find_all('span', class_='float-right')
      if gen is not None:
        # manufacturer
        try:
          manufa = gen[0]
          if manufa is not None:
            manufac = manufa.text.strip()
            manufacturer.append(manufac)
          else:
            manufac = na
            manufacturer.append(manufac)
        except IndexError:
          manufac = na
          manufacturer.append(manufac)
        
        time.sleep(0.15)

        # model
        try:
          m = gen[1]
          if m is not None:
            mod = m.text.strip()
            model.append(mod)
          else:
            mod = na
            model.append(mod)
        except IndexError:
          mod = na
          model.append(mod)

        time.sleep(0.15)

        # year
        try:
          y = gen[2]
          if y is not None:
            yr = y.text.strip()
            year.append(yr)
          else:
            yr = na
            year.append(yr)
        except IndexError:
          yr = na
          year.append(yr)

        time.sleep(0.15)

         # mileage
        try:
          mil = gen[3]
          if mil is not None:
            mile = mil.text.strip()
            mileage.append(mile)
          else:
            mile = na
            mileage.append(mile)
        except IndexError:
          mile = na
          mileage.append(mile)

        time.sleep(0.15)

        # transmission
        try:
          tra = gen[5]
          if tra is not None:
            trans = tra.text.strip()
            transmission.append(trans)
          else:
            trans = na
            transmission.append(trans)
        except IndexError:
          trans = na
          transmission.append(trans)

        time.sleep(0.15)

        # color
        try:
          co = gen[7]
          if co is not None:
            col = co.text.strip()
            color.append(col)
          else:
            col = na
            color.append(col)
        except IndexError:
          col = na
          color.append(col)

        time.sleep(0.15)
      
      else:
        manufac = na
        manufacturer.append(manufac)
        mod = na
        model.append(mod)
        yr = na
        year.append(yr)
        mile = na
        mileage.append(mile)
        trans = na
        transmission.append(trans)
        col = na
        color.append(col)

      time.sleep(1.5)
  
    except AttributeError:
      car_err_urls.append(i)
  

In [ ]:
# check the length of each array gotten
for i in [manufacturer, model, price, year, mileage, grade, selling_condition, fuel, transmission, color]:
  print(len(i))

4167
4167
4167
4167
4167
4167
4167
4167
4167
4167


In [ ]:
err_urls = car_err_urls.copy()

In [ ]:
len(err_urls)

16

In [ ]:
# Scraping the remaining cars from error urls
na = np.nan
# manufacturer = []
# model = []
# year = []
# price = []
# mileage = []
# grade = []
# selling_condition = []
# transmission = []
# color = []
# fuel = []

car_err_urls = []

car = 0
for i in tqdm(err_urls):
  car += 1
  print(f'car {car}')
  page = requests.get(i).text
  soup = bs4(page, 'html.parser')

  # error check to ensure the car isn't sold out
  if soup.get_text()[11:16] == 'Sorry':
    print('car is sold out. No details available')
  else:
    try:
      # price
      pr = soup.find(class_='price_title amount mt-1').text.strip()
      pri = pr.replace('₦', '').replace(',', '').lstrip()
      price.append(pri)
      
      time.sleep(0.2)

      # selling condition
      con = soup.find(class_='justify-content-center w-100')
      if con is not None:
        cond = con.text.strip()
        selling_condition.append(cond)
      else:
        cond = na
        selling_condition.append(cond)

      time.sleep(0.15)

      # fuel
      f = soup.find(class_='overview_icon_box clearfix')
      if f is not None:
        fu = f.figcaption
        if fu is not None:
          fuu = fu.text.strip()
          fuel.append(fuu)
        else:
          fu = na
          fuel.append(fu)
      else:
        fu = na
        fuel.append(fu)
    
      time.sleep(0.15)

      # grade
      g = soup.find_all('div', class_='col-sm-6 col-md-3 col-6')
      try:
        gr = g[3]['title']
        if gr is not None:
          grr = gr[5]
          grade.append(grr)
        else:
          gr = na
          grade.append(gr)
      except IndexError:
        gr = na
        grade.append(gr)
        
      time.sleep(0.15)

      # remaining features
      gen = soup.find_all('span', class_='float-right')
      if gen is not None:
        # manufacturer
        try:
          manufa = gen[0]
          if manufa is not None:
            manufac = manufa.text.strip()
            manufacturer.append(manufac)
          else:
            manufac = na
            manufacturer.append(manufac)
        except IndexError:
          manufac = na
          manufacturer.append(manufac)
        
        time.sleep(0.15)

        # model
        try:
          m = gen[1]
          if m is not None:
            mod = m.text.strip()
            model.append(mod)
          else:
            mod = na
            model.append(mod)
        except IndexError:
          mod = na
          model.append(mod)

        time.sleep(0.15)

        # year
        try:
          y = gen[2]
          if y is not None:
            yr = y.text.strip()
            year.append(yr)
          else:
            yr = na
            year.append(yr)
        except IndexError:
          yr = na
          year.append(yr)

        time.sleep(0.15)

         # mileage
        try:
          mil = gen[3]
          if mil is not None:
            mile = mil.text.strip()
            mileage.append(mile)
          else:
            mile = na
            mileage.append(mile)
        except IndexError:
          mile = na
          mileage.append(mile)

        time.sleep(0.15)

        # transmission
        try:
          tra = gen[5]
          if tra is not None:
            trans = tra.text.strip()
            transmission.append(trans)
          else:
            trans = na
            transmission.append(trans)
        except IndexError:
          trans = na
          transmission.append(trans)

        time.sleep(0.15)

        # color
        try:
          co = gen[7]
          if co is not None:
            col = co.text.strip()
            color.append(col)
          else:
            col = na
            color.append(col)
        except IndexError:
          col = na
          color.append(col)

        time.sleep(0.15)
      
      else:
        manufac = na
        manufacturer.append(manufac)
        mod = na
        model.append(mod)
        yr = na
        year.append(yr)
        mile = na
        mileage.append(mile)
        trans = na
        transmission.append(trans)
        col = na
        color.append(col)

      time.sleep(1.5)
  
    except AttributeError:
      car_err_urls.append(i)
  


  0%|          | 0/16 [00:00<?, ?it/s]

car 1



  6%|▋         | 1/16 [00:04<01:04,  4.28s/it]

car 2



 12%|█▎        | 2/16 [00:08<00:59,  4.22s/it]

...


 88%|████████▊ | 14/16 [00:57<00:08,  4.14s/it]

car 15



 94%|█████████▍| 15/16 [01:02<00:04,  4.18s/it]

car 16



100%|██████████| 16/16 [01:05<00:00,  4.12s/it]


In [ ]:
for i in [manufacturer, model, price, year, mileage, grade, selling_condition, fuel, transmission, color]:
  print(len(i))

4183
4183
4183
4183
4183
4183
4183
4183
4183
4183


In [ ]:
len(car_err_urls)

0

No more error urls. 

We put each array into a dataframe

In [ ]:
d = {'manufacturer':manufacturer, 'model':model, 'year':year, 'mileage':mileage, 'selling_condition':selling_condition, 'grade':grade, 'fuel':fuel, 'transmission':transmission, 'color':color, 'price':price}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))

In [ ]:
df

,manufacturer,model,year,mileage,selling_condition,grade,fuel,transmission,color,price
0,Toyota,Land Cruiser,2020,2197,Nigerian used,,Petrol,Automatic,Black,33000000
1,Acura,Mdx,2008,105242,Nigerian used,D,Petrol,Automatic,GREY,2586000
2,Toyota,Highlander,2001,152321,Nigerian used,C,Petrol,Automatic,GREEN,1914000
3,Ford,Explorer,2014,60187,Nigerian used,C,Petrol,Automatic,BROWN,4154000
4,Ford,Fusion,2012,81802,Nigerian used,,Petrol,Automatic,NaN,1018000
...,...,...,...,...,...,...,...,...,...,...
4178,Land Rover,Range Rover Sport,2006,150104,Foreign used,C,Petrol,Automatic,NaN,3930000
4179,Ford,Escape,2003,137494,Nigerian used,E,Petrol,Automatic,Aqua Green,682000
4180,Lexus,Rx 350,2008,75437,Nigerian used,D,Petrol,Automatic,GREEN,3146000
4181,Ford,Edge,2010,187545,Foreign used,,Petrol,Automatic,Black,4600000


In [ ]:
# save to csv
df.to_csv('final_scrape_cars45.csv', index=False)